**# Step- 1: Import libraries**

In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

# Step- 2: Fetch current weather data

In [ ]:
API_KEY= "1798b45eacad952fe5736f95bba1073c"
BASE_URL = "https://api.openweathermap.org/data/2.5/"

In [ ]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    if response.status_code != 200:
        print(f"Error: {data['message']}")
        return None
    return{
        'city' : data['name'],
        'current_temp' : round(data['main']['temp']),
        'feels_like' : round(data['main']['feels_like']),
        'temp_min' : round(data['main']['temp_min']),
        'temp_max' : round(data['main']['temp_max']),
        'humidity' : round(data['main']['humidity']),
        'description' : data['weather'][0]['description'],
        'country' : data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'Wind_Gust_speed': data['wind']['speed']
    }

## Step-3: Read historical data

In [ ]:
def read_historical_data(filename):
    df = pd.read_csv(filename)  # load CSV file into DataFrame
    df = df.dropna()  # remove rows with missing values
    df = df.drop_duplicates()  # remove duplicate rows
    return df


## Step-4: Prepare data for training

In [ ]:
def prepare_data(data):
    le = LabelEncoder()  # create a LabelEncoder instance

    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

    # define the feature variable and target variables
    X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]  # features
    y = data['RainTomorrow']  # target variable

    return X, y, le  # return feature variable, target variable, and the label encoder


## Step-5: Train Rain Prediction Model

In [ ]:
def train_rain_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)  # train the model

    y_pred = model.predict(X_test)  # to make predictions on test set

    print("Mean Squared Error for Rain Model")
    print(mean_squared_error(y_test, y_pred))

    return model

## Step-6: Prepare Regression data

In [ ]:
def prepare_regression_data(data, feature):
    X, y = [], []  # initialize list for feature and target values

    for i in range(len(data) - 1):
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i + 1])

    X = np.array(X).reshape(-1, 1)
    y = np.array(y)
    return X, y

## Step-7: Train Regression model

In [ ]:
def train_regression_model(X,y):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X,y)
    return model

## Step-8: Predict Future

In [ ]:
def predict_future(model, current_value):
    prediction = [current_value]

    for i in range(5):
        next_value = model.predict(np.array([[prediction[-1]]]))

        prediction.append(next_value[0])

    return prediction[1:]

## Step-9: Weather Analysis Function

In [ ]:
def weather_view():
    city = input("Enter any city name:")
    current_weather = get_current_weather(city)

    historical_data = read_historical_data("/content/weather.csv")

    # Define all possible wind directions from compass points
    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
    ]
    all_wind_directions = [point[0] for point in compass_points]

    # Prepare historical data and train the rain model
    historical_data['WindGustDir'] = historical_data['WindGustDir'].replace('WNW ', 'WNW') # Fix potential typo
    X, y, le = prepare_data(historical_data)

    # Update the label encoder classes to include all possible wind directions
    le.classes_ = np.array(all_wind_directions)

    rain_model = train_rain_model(X, y)


    # map wind direction to compass points
    wind_deg = current_weather['wind_gust_dir'] % 360


    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)[0]

    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction,
        'WindGustSpeed': current_weather['Wind_Gust_speed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp']
    }

    current_df = pd.DataFrame([current_data])

    # Encode 'WindGustDir' in current_df using the same label encoder
    current_df['WindGustDir'] = le.transform(current_df['WindGustDir'])

    # rain prediction
    rain_prediction = rain_model.predict(current_df)[0]

    # prepare regression model for temperature and humidity
    x_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(x_temp, y_temp)
    hum_model = train_regression_model(x_hum, y_hum)

    # predict future temperature and humidity
    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    # prepare time for future predictions
    timezone = pytz.timezone('Asia/Karachi')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)


    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:%M") for i in range(5)]

    # Display results
    print(f"City: {city}, {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}")
    print(f"Feels Like: {current_weather['feels_like']}")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")


    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Predictions:")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")


weather_view()

Enter any city name:Ghaziabad
Mean Squared Error for Rain Model
0.1506849315068493
City: Ghaziabad, IN
Current Temperature: 32
Feels Like: 32
Minimum Temperature: 32°C
Maximum Temperature: 32°C
Humidity: 40%
Weather Prediction: clear sky
Rain Prediction: No

Future Temperature Predictions:
10:00: 32.5°C
11:00: 32.0°C
12:00: 32.4°C
13:00: 33.0°C
14:00: 25.1°C

Future Humidity Predictions:
10:00: 45.9%
11:00: 48.2%
12:00: 53.7%
13:00: 58.1%
14:00: 54.2%
